In [1]:
import nb_setup 
import importlib
importlib.reload(nb_setup)
nb_setup.init()

Added project root to sys.path: /Users/falcon/Developments/act/backtesting-playground/trading
Changed current working directory to: /Users/falcon/Developments/act/backtesting-playground/trading
Initialized project with base directory: /Users/falcon/Developments/act/backtesting-playground/trading


In [2]:
import models.market as market
import importlib
importlib.reload(market)

market = market.Market.from_json_file()
df = market.to_df()
df.head()

,maxLeverage,name,onlyIsolated,szDecimals,dayNtlVlm,funding,impactPxs,markPx,midPx,openInterest,oraclePx,premium,prevDayPx
0,50,BTC,False,5,3.765439e+08,-0.000008,"[64274.0, 64275.0]",64276.00000,64274.50000,1.810208e+03,64313.00000,-0.000591,66769.00000
1,50,ETH,False,4,3.390759e+08,0.000013,"[3157.4, 3157.64]",3157.40000,3157.45000,3.269658e+04,3158.20000,-0.000177,3259.80000
2,20,ATOM,False,2,2.622425e+06,0.000013,"[8.4329, 8.4341]",8.43340,8.43350,6.048372e+04,8.44000,-0.000699,8.83850
3,20,MATIC,False,1,6.717067e+06,0.000013,"[0.71067, 0.71091]",0.71061,0.71071,2.202128e+06,0.71095,-0.000056,0.75204
4,20,DYDX,False,1,1.279112e+06,0.000013,"[2.18916, 2.19116]",2.19000,2.18970,5.700630e+05,2.19200,-0.000383,2.42690


In [3]:
# get top n coins

def get_top_n_coins(n=60):
    top_n_coins_df = df.sort_values(by='dayNtlVlm', ascending=False).head(n)
    top_n_coins_df = top_n_coins_df.reset_index(drop=True)
    top_n_coins_df.head()

    # get first coin name
    top_n_coins = top_n_coins_df.get('name')
    first_coin = top_n_coins[0]
    print("top 1st coin:", first_coin)
    return top_n_coins

top_n_coins = get_top_n_coins(60)

top 1st coin: BTC


In [5]:
%time
import time
from hyperliquid.info import Info
from hyperliquid.utils import constants as hl_constants
import utils.now_timestamp as now_timestamp 
import utils.fetch_coin as fetch_coin
import models.candles as candles
import importlib
importlib.reload(fetch_coin)
importlib.reload(candles)
importlib.reload(now_timestamp)

info = Info(hl_constants.MAINNET_API_URL, skip_ws=True)
def fetch_coins_by_names(names):
    start = now_timestamp.get_timestamp_from_today_midnight_utc(days=-30)
    end = now_timestamp.get_timestamp_from_today_midnight_utc(days=0)
    error_coins = []
    for coin in names:
        try:
            time.sleep(0.2)
            df = fetch_coin.fetch_coin(
                coin,
                interval='4h',
                bar_count=380
            )
        except Exception as e:
            print(f"coin {coin}: not fetched")
            print(e)
            error_coins.append(coin)
            continue

fetch_coins_by_names(top_n_coins)
# fetch_coins_by_names(["BTC"])

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.11 µs
BTC
ETH
SOL
kPEPE
WIF
NEAR
kBONK
BNB
ARB
DOGE
TIA
AVAX
FTM
WLD
SUI
OP
ONDO
APT
FET
MATIC
INJ
ENA
SEI
MKR
PENDLE
RNDR
BCH
ORDI
LINK
STX
HBAR
kSHIB
JUP
LTC
TON
AR
JTO
RUNE
LDO
BLUR
FIL
ATOM
XRP
CFX
STRK
kFLOKI
ADA
BOME
DOT
NEO
PIXEL
TNSR
APE
MAVIA
CAKE
UNI
PYTH
MERL
ENS
AI
